# 1. Install & Import 라이브러리

In [1]:
!pip3 install selenium webdriver_manager beautifulsoup4 pandas

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# 2. 경로 설정

In [2]:
PATH_POST_DATA_CSV = 'data_csv/post.csv'
PATH_MATCH_STAT_CSV = 'data_csv/match_stats.csv'
PATH_PLAYERS_TXT = 'data_txt/players.txt'
PATH_URLS_TXT = 'data_txt/URLs.txt'

# 3. 데이터 수집

In [ ]:
# 크롤링할 URL 파일 불러오기 (소집명단 발표 단위 선수 명단)
with open(PATH_URLS_TXT, 'r') as f:
    url_box = set()
    for line in f.readlines():
        url_box.add(line)

# KFA사이트 크롤링 -> A매치 국가대표 선수 리스트 생성
player_box = set()
def crawling_data(url):
    # 웹 드라이버 초기화
    options = webdriver.ChromeOptions()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # 페이지 접속
    try:
        driver.get(url)
        sleep(3)
    except Exception as e: 
        print(e)

    # BeautifulSoup 객체 생성
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    # 선수 이름 추출
    for p in soup.find_all('p', {'class':'pic'}):
        try:
            name = p.text.strip().split()[0]
            player_box.add(name)
        except Exception:
            pass

# TXT 파일(선수 리스트)이 없으면 크롤링 시작
if os.path.exists(PATH_PLAYERS_TXT):
    # TXT 파일이 이미 존재하면 파일에서 데이터 읽기
    with open(PATH_PLAYERS_TXT, 'r', encoding='utf-8') as f:
        for p in f.readlines():
            player_box.add(p.strip())
else:
    for i, url in enumerate(url_box):
        # 진행률
        progress = f'{100*i/len(url_box)}'
        print(f'\r{progress}%, ({i}/{len(url_box)})', end = '')

        # 크롤링
        try:
            crawling_data(url)
        except Exception as e:
            print(e)
            
    # 수집한 데이터 TXT파일로 저장
    for p in player_box:
        with open(PATH_PLAYERS_TXT, 'a', encoding='utf-8') as f:
            f.write(p+'\n')
            